In [4]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
def fetch_hotel_data(url):
    headers = {"User-Agent": "Student_musashinodaigaku_2522044"}
    response = requests.get(url, headers=headers)
    
    # 解析の開始
    soup = BeautifulSoup(response.content, 'html.parser')
    hotels = []

    # アパホテルの検索結果は各ホテルが特定のクラスを持つdivに囲まれています
    # ※実際のクラス名はサイト更新で変わる可能性があるため、ブラウザの検証ツールで確認してください
    hotel_elements = soup.find_all('div', class_='p-hotelList_item') 

    for element in hotel_elements:
        # ホテル名の取得
        name = element.find('h2', class_='p-hotelList_item_name').text.strip()
        
        # 最安値の取得（例）
        price_text = element.find('span', class_='p-hotelList_item_priceValue').text.replace(',', '')
        price = int(price_text)
        
        hotels.append({'name': name, 'price': price})

    time.sleep(1.5) # 負荷配慮 [cite: 37]
    return hotels

def save_to_db(hotel_list, date_label):
    conn = sqlite3.connect('hotel_analysis.db')
    cursor = conn.cursor()
    
    # テーブル作成（宿泊日ラベル、ホテル名、価格） [cite: 33]
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS hotel_prices (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            target_date TEXT,
            hotel_name TEXT,
            price INTEGER
        )
    ''')
    
    for hotel in hotel_list:
        cursor.execute(
            'INSERT INTO hotel_prices (target_date, hotel_name, price) VALUES (?, ?, ?)',
            (date_label, hotel['name'], hotel['price'])
        )
    
    conn.commit()
    conn.close()